# AWS EC2 AMI Or Snapshot Shared Publicly

Detect when an Amazon EC2 AMI or Snapshot is shared publicly. Public sharing can lead to unauthorised access and potential data exfiltration.

## Detection Query
Execute the following query to find alerts related to this rule.

In [ ]:
%ingest.source_type="aws:cloudtrail"
eventSource="ec2.amazonaws.com"
eventName:(
"ModifyImageAttribute"
"ModifySnapshotAttribute"
)
(
requestParameters.launchPermission.add.items.group="all"
or requestParameters.createVolumePermission.add.items.group="all"
)
| groupbycount(
recipientAccountId,
awsRegion,
eventName,
requestParameters.imageId,
requestParameters.snapshotId,
userIdentity.type,
userIdentity.arn,
sourceIPAddress
)
| where @q.count > 0

## Investigation Steps
1. **Identify the Principal**: Who performed the action?
2. **Review Context**: What else did this principal do?
3. **Check Permissions**: Did the principal have legitimate reasons for this action?
4. **Verify Resource**: Is the resource critical or sensitive?

In [ ]:
%ingest.source_type:aws:cloudtrail
userIdentity.arn: <PRINCIPAL_ARN>
| count(eventName) by eventName